In [98]:
import psycopg2
import pandas as pd
import numpy as np
import pickle as pkl
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt

pd.set_option('display.max_columns',300)
pd.set_option('display.max_rows',300)

In [2]:
host = '119.8.167.24'
port = '5432'
username = 'team27'
password = 'bill-mirror-nerve27'
database = 'mimiciv' #mimic

In [3]:
con = psycopg2.connect(database=database, user=username, password=password, host=host, port=port)

In [4]:
cur = con.cursor()

In [5]:
mode = 'load' # 'query' or 'load'

# Hospital and ICU cohort

In [11]:
%%time

if mode == 'query':

    query = '''
    select 

    adm_patient.gender,
    adm_patient.anchor_age,
    adm_patient.dod,
    admittime, 
    dischtime, 
    deathtime, 
    admission_type, 
    admission_location, 
    discharge_location, 
    insurance,edregtime, 
    edouttime,
    hospital_expire_flag,
    first_icu.*,
    DATE_PART('day',dischtime - admittime)+1 as hosp_los

    from
    (
        select *
            from mimic_core.admissions adm
        join 
            mimic_core.patients pat
        on pat.subject_id = adm.subject_id
        where pat.anchor_age >= 18
        and adm.hospital_expire_flag = 0 

      ) adm_patient

    join

    (
        select icu.*
            from mimic_icu.icustays icu
        join
        (
            select hadm_id  , min(intime) as min_intime 
            FROM 
                mimic_icu.icustays 
            GROUP BY hadm_id 
        ) first_
        on icu.hadm_id = first_.hadm_id and icu.intime = first_.min_intime

    ) first_icu
    on adm_patient.hadm_id = first_icu.hadm_id


    '''

    hosp_icu = pd.read_sql(query, con)

In [6]:
if mode == 'query':
    with open('hosp_icu.pkl', 'wb') as f:
        pkl.dump(hosp_icu, f)
elif mode == 'load':
    with open('hosp_icu.pkl', 'rb') as f:
        hosp_icu = pkl.load(f)

In [7]:
hosp_icu = hosp_icu.rename(columns={'los':'icu_los'})

In [8]:
hosp_icu.shape

(58101, 22)

In [9]:
hosp_icu.head()

,gender,anchor_age,dod,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,edregtime,edouttime,hospital_expire_flag,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,icu_los,hosp_los
0,M,71,None,2121-08-30 16:33:00,2121-09-03 15:45:00,None,URGENT,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Medicare,NaT,NaT,0,14990224,20000147,37348463,Cardiac Vascular Intensive Care Unit (CVICU),Cardiac Vascular Intensive Care Unit (CVICU),2121-08-30 18:08:39,2121-08-31 21:29:49,1.139699,3.0
1,M,63,None,2125-10-26 00:00:00,2125-10-28 19:25:00,None,SURGICAL SAME DAY ADMISSION,PHYSICIAN REFERRAL,HOME,Medicaid,NaT,NaT,0,15975141,20001494,39346951,Neuro Intermediate,Neuro Intermediate,2125-10-26 17:34:33,2125-10-28 19:33:26,2.082558,2.0
2,M,64,None,2196-06-12 00:00:00,2196-06-19 20:01:00,None,ELECTIVE,PHYSICIAN REFERRAL,HOME HEALTH CARE,Medicare,NaT,NaT,0,17112572,20001687,32926753,Cardiac Vascular Intensive Care Unit (CVICU),Cardiac Vascular Intensive Care Unit (CVICU),2196-06-13 09:18:36,2196-06-15 10:34:07,2.052442,7.0
3,M,56,None,2157-09-23 23:24:00,2157-09-29 14:20:00,None,EW EMER.,EMERGENCY ROOM,SKILLED NURSING FACILITY,Medicare,2157-09-23 18:09:00,2157-09-24 01:01:00,0,18826698,20002252,35781968,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-09-24 01:01:00,2157-09-24 15:32:48,0.605417,5.0
4,M,46,None,2156-08-31 14:52:00,2156-09-03 11:15:00,None,URGENT,TRANSFER FROM HOSPITAL,HOME,Other,NaT,NaT,0,18346781,20002270,38835257,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2156-08-31 14:52:54,2156-09-01 02:07:59,0.468808,2.0


# Ventilator queries

In [99]:
%%time
query = '''
SELECT subject_id, stay_id, charttime FROM (
  select vs.subject_id, ce.stay_id, vs.charttime, vs.ventilator_mode, vs.extubated from mimic_derived.ventilator_setting vs 
  JOIN (select subject_id, stay_id, charttime from mimic_icu.chartevents ) ce
  on  vs.subject_id = ce.subject_id and vs.charttime = ce.charttime
  where extubated = 1) vsce
  GROUP BY subject_id, stay_id, charttime
'''

vs = pd.read_sql(query, con)

query = '''
select *, DATE_PART('day', endtime - starttime)+1 as vent_duration from mimic_derived.ventilator_durations vd
'''

vd = pd.read_sql(query, con)

Wall time: 584 ms


In [10]:
if mode == 'query':
    with open('vs.pkl', 'wb') as f:
        pkl.dump(vs, f)
    with open('vd.pkl', 'wb') as f:
        pkl.dump(vd, f)   
elif mode == 'load':
    with open('vs.pkl', 'rb') as f:
        vs = pkl.load(f)
    with open('vd.pkl', 'rb') as f:
        vd = pkl.load(f)

In [11]:
vs.shape, vd.shape

((6161, 3), (28908, 5))

In [12]:
vsvd = pd.merge(vs, vd, on=['stay_id'])

In [13]:
vsvd[vsvd['subject_id']==16108683]

,subject_id,stay_id,charttime,ventnum,starttime,endtime,vent_duration
3,16108683,38197814,2120-11-21 16:10:00,1,2120-11-10 04:00:00,2120-11-12 16:09:00,3.0
4,16108683,38197814,2120-11-21 16:10:00,2,2120-11-12 23:00:00,2120-11-21 16:10:00,9.0
5,16108683,38197814,2120-11-21 16:10:00,3,2120-11-22 02:12:00,2120-11-27 18:00:00,6.0
6,16108683,38197814,2120-11-27 18:00:00,1,2120-11-10 04:00:00,2120-11-12 16:09:00,3.0
7,16108683,38197814,2120-11-27 18:00:00,2,2120-11-12 23:00:00,2120-11-21 16:10:00,9.0
8,16108683,38197814,2120-11-27 18:00:00,3,2120-11-22 02:12:00,2120-11-27 18:00:00,6.0


In [14]:
vsvd1 = pd.merge(vs, vd, left_on=['stay_id','charttime'], right_on=['stay_id','endtime'])

In [15]:
vsvd1[vsvd1['subject_id']==16108683]

,subject_id,stay_id,charttime,ventnum,starttime,endtime,vent_duration
3,16108683,38197814,2120-11-21 16:10:00,2,2120-11-12 23:00:00,2120-11-21 16:10:00,9.0
2727,16108683,38197814,2120-11-27 18:00:00,3,2120-11-22 02:12:00,2120-11-27 18:00:00,6.0


In [16]:
vsvd1 = vsvd1.groupby(['subject_id','stay_id']).agg({'vent_duration':'sum'}).reset_index()

In [17]:
vsvd1[vsvd1['subject_id']==16108683]

,subject_id,stay_id,vent_duration
2938,16108683,38197814,15.0


In [18]:
hosp_icu[hosp_icu['stay_id']==38197814]

,gender,anchor_age,dod,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,edregtime,edouttime,hospital_expire_flag,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,icu_los,hosp_los
16964,M,41,None,2120-11-08 15:02:00,2121-03-12 14:40:00,None,URGENT,TRANSFER FROM HOSPITAL,REHAB,Medicaid,NaT,NaT,0,16108683,21607477,38197814,Coronary Care Unit (CCU),Medical/Surgical Intensive Care Unit (MICU/SICU),2120-11-10 03:56:33,2120-11-29 17:46:11,19.576134,123.0


## CCI

In [8]:
%%time # Wall time: 2min 5s

if mode == 'query':

    query = '''
    WITH diag AS
    (
        SELECT 
            hadm_id
            , CASE WHEN icd_version = 9 THEN icd_code ELSE NULL END AS icd9_code
            , CASE WHEN icd_version = 10 THEN icd_code ELSE NULL END AS icd10_code
        FROM mimic_hosp.diagnoses_icd diag
    )
    , com AS
    (
        SELECT
            ad.hadm_id

            -- Myocardial infarction
            , MAX(CASE WHEN
                SUBSTR(icd9_code, 1, 3) IN ('410','412')
                OR
                SUBSTR(icd10_code, 1, 3) IN ('I21','I22')
                OR
                SUBSTR(icd10_code, 1, 4) = 'I252'
                THEN 1 
                ELSE 0 END) AS myocardial_infarct

            -- Congestive heart failure
            , MAX(CASE WHEN 
                SUBSTR(icd9_code, 1, 3) = '428'
                OR
                SUBSTR(icd9_code, 1, 5) IN ('39891','40201','40211','40291','40401','40403',
                              '40411','40413','40491','40493')
                OR 
                SUBSTR(icd9_code, 1, 4) BETWEEN '4254' AND '4259'
                OR
                SUBSTR(icd10_code, 1, 3) IN ('I43','I50')
                OR
                SUBSTR(icd10_code, 1, 4) IN ('I099','I110','I130','I132','I255','I420',
                                                       'I425','I426','I427','I428','I429','P290')
                THEN 1 
                ELSE 0 END) AS congestive_heart_failure

            -- Peripheral vascular disease
            , MAX(CASE WHEN 
                SUBSTR(icd9_code, 1, 3) IN ('440','441')
                OR
                SUBSTR(icd9_code, 1, 4) IN ('0930','4373','4471','5571','5579','V434')
                OR
                SUBSTR(icd9_code, 1, 4) BETWEEN '4431' AND '4439'
                OR
                SUBSTR(icd10_code, 1, 3) IN ('I70','I71')
                OR
                SUBSTR(icd10_code, 1, 4) IN ('I731','I738','I739','I771','I790',
                                                       'I792','K551','K558','K559','Z958','Z959')
                THEN 1 
                ELSE 0 END) AS peripheral_vascular_disease

            -- Cerebrovascular disease
            , MAX(CASE WHEN 
                SUBSTR(icd9_code, 1, 3) BETWEEN '430' AND '438'
                OR
                SUBSTR(icd9_code, 1, 5) = '36234'
                OR
                SUBSTR(icd10_code, 1, 3) IN ('G45','G46')
                OR 
                SUBSTR(icd10_code, 1, 3) BETWEEN 'I60' AND 'I69'
                OR
                SUBSTR(icd10_code, 1, 4) = 'H340'
                THEN 1 
                ELSE 0 END) AS cerebrovascular_disease

            -- Dementia
            , MAX(CASE WHEN 
                SUBSTR(icd9_code, 1, 3) = '290'
                OR
                SUBSTR(icd9_code, 1, 4) IN ('2941','3312')
                OR
                SUBSTR(icd10_code, 1, 3) IN ('F00','F01','F02','F03','G30')
                OR
                SUBSTR(icd10_code, 1, 4) IN ('F051','G311')
                THEN 1 
                ELSE 0 END) AS dementia

            -- Chronic pulmonary disease
            , MAX(CASE WHEN 
                SUBSTR(icd9_code, 1, 3) BETWEEN '490' AND '505'
                OR
                SUBSTR(icd9_code, 1, 4) IN ('4168','4169','5064','5081','5088')
                OR 
                SUBSTR(icd10_code, 1, 3) BETWEEN 'J40' AND 'J47'
                OR 
                SUBSTR(icd10_code, 1, 3) BETWEEN 'J60' AND 'J67'
                OR
                SUBSTR(icd10_code, 1, 4) IN ('I278','I279','J684','J701','J703')
                THEN 1 
                ELSE 0 END) AS chronic_pulmonary_disease

            -- Rheumatic disease
            , MAX(CASE WHEN 
                SUBSTR(icd9_code, 1, 3) = '725'
                OR
                SUBSTR(icd9_code, 1, 4) IN ('4465','7100','7101','7102','7103',
                                                      '7104','7140','7141','7142','7148')
                OR
                SUBSTR(icd10_code, 1, 3) IN ('M05','M06','M32','M33','M34')
                OR
                SUBSTR(icd10_code, 1, 4) IN ('M315','M351','M353','M360')
                THEN 1 
                ELSE 0 END) AS rheumatic_disease

            -- Peptic ulcer disease
            , MAX(CASE WHEN 
                SUBSTR(icd9_code, 1, 3) IN ('531','532','533','534')
                OR
                SUBSTR(icd10_code, 1, 3) IN ('K25','K26','K27','K28')
                THEN 1 
                ELSE 0 END) AS peptic_ulcer_disease

            -- Mild liver disease
            , MAX(CASE WHEN 
                SUBSTR(icd9_code, 1, 3) IN ('570','571')
                OR
                SUBSTR(icd9_code, 1, 4) IN ('0706','0709','5733','5734','5738','5739','V427')
                OR
                SUBSTR(icd9_code, 1, 5) IN ('07022','07023','07032','07033','07044','07054')
                OR
                SUBSTR(icd10_code, 1, 3) IN ('B18','K73','K74')
                OR
                SUBSTR(icd10_code, 1, 4) IN ('K700','K701','K702','K703','K709','K713',
                                                       'K714','K715','K717','K760','K762',
                                                       'K763','K764','K768','K769','Z944')
                THEN 1 
                ELSE 0 END) AS mild_liver_disease

            -- Diabetes without chronic complication
            , MAX(CASE WHEN 
                SUBSTR(icd9_code, 1, 4) IN ('2500','2501','2502','2503','2508','2509') 
                OR
                SUBSTR(icd10_code, 1, 4) IN ('E100','E10l','E106','E108','E109','E110','E111',
                                                       'E116','E118','E119','E120','E121','E126','E128',
                                                       'E129','E130','E131','E136','E138','E139','E140',
                                                       'E141','E146','E148','E149')
                THEN 1 
                ELSE 0 END) AS diabetes_without_cc

            -- Diabetes with chronic complication
            , MAX(CASE WHEN 
                SUBSTR(icd9_code, 1, 4) IN ('2504','2505','2506','2507')
                OR
                SUBSTR(icd10_code, 1, 4) IN ('E102','E103','E104','E105','E107','E112','E113',
                                                       'E114','E115','E117','E122','E123','E124','E125',
                                                       'E127','E132','E133','E134','E135','E137','E142',
                                                       'E143','E144','E145','E147')
                THEN 1 
                ELSE 0 END) AS diabetes_with_cc

            -- Hemiplegia or paraplegia
            , MAX(CASE WHEN 
                SUBSTR(icd9_code, 1, 3) IN ('342','343')
                OR
                SUBSTR(icd9_code, 1, 4) IN ('3341','3440','3441','3442',
                                                      '3443','3444','3445','3446','3449')
                OR 
                SUBSTR(icd10_code, 1, 3) IN ('G81','G82')
                OR 
                SUBSTR(icd10_code, 1, 4) IN ('G041','G114','G801','G802','G830',
                                                       'G831','G832','G833','G834','G839')
                THEN 1 
                ELSE 0 END) AS paraplegia

            -- Renal disease
            , MAX(CASE WHEN 
                SUBSTR(icd9_code, 1, 3) IN ('582','585','586','V56')
                OR
                SUBSTR(icd9_code, 1, 4) IN ('5880','V420','V451')
                OR
                SUBSTR(icd9_code, 1, 4) BETWEEN '5830' AND '5837'
                OR
                SUBSTR(icd9_code, 1, 5) IN ('40301','40311','40391','40402','40403','40412','40413','40492','40493')          
                OR
                SUBSTR(icd10_code, 1, 3) IN ('N18','N19')
                OR
                SUBSTR(icd10_code, 1, 4) IN ('I120','I131','N032','N033','N034',
                                                       'N035','N036','N037','N052','N053',
                                                       'N054','N055','N056','N057','N250',
                                                       'Z490','Z491','Z492','Z940','Z992')
                THEN 1 
                ELSE 0 END) AS renal_disease

            -- Any malignancy, including lymphoma and leukemia, except malignant neoplasm of skin
            , MAX(CASE WHEN 
                SUBSTR(icd9_code, 1, 3) BETWEEN '140' AND '172'
                OR
                SUBSTR(icd9_code, 1, 4) BETWEEN '1740' AND '1958'
                OR
                SUBSTR(icd9_code, 1, 3) BETWEEN '200' AND '208'
                OR
                SUBSTR(icd9_code, 1, 4) = '2386'
                OR
                SUBSTR(icd10_code, 1, 3) IN ('C43','C88')
                OR
                SUBSTR(icd10_code, 1, 3) BETWEEN 'C00' AND 'C26'
                OR
                SUBSTR(icd10_code, 1, 3) BETWEEN 'C30' AND 'C34'
                OR
                SUBSTR(icd10_code, 1, 3) BETWEEN 'C37' AND 'C41'
                OR
                SUBSTR(icd10_code, 1, 3) BETWEEN 'C45' AND 'C58'
                OR
                SUBSTR(icd10_code, 1, 3) BETWEEN 'C60' AND 'C76'
                OR
                SUBSTR(icd10_code, 1, 3) BETWEEN 'C81' AND 'C85'
                OR
                SUBSTR(icd10_code, 1, 3) BETWEEN 'C90' AND 'C97'
                THEN 1 
                ELSE 0 END) AS malignant_cancer

            -- Moderate or severe liver disease
            , MAX(CASE WHEN 
                SUBSTR(icd9_code, 1, 4) IN ('4560','4561','4562')
                OR
                SUBSTR(icd9_code, 1, 4) BETWEEN '5722' AND '5728'
                OR
                SUBSTR(icd10_code, 1, 4) IN ('I850','I859','I864','I982','K704','K711',
                                                       'K721','K729','K765','K766','K767')
                THEN 1 
                ELSE 0 END) AS severe_liver_disease

            -- Metastatic solid tumor
            , MAX(CASE WHEN 
                SUBSTR(icd9_code, 1, 3) IN ('196','197','198','199')
                OR 
                SUBSTR(icd10_code, 1, 3) IN ('C77','C78','C79','C80')
                THEN 1 
                ELSE 0 END) AS metastatic_solid_tumor

            -- AIDS/HIV
            , MAX(CASE WHEN 
                SUBSTR(icd9_code, 1, 3) IN ('042','043','044')
                OR 
                SUBSTR(icd10_code, 1, 3) IN ('B20','B21','B22','B24')
                THEN 1 
                ELSE 0 END) AS aids
        FROM mimic_core.admissions ad
        LEFT JOIN diag
        ON ad.hadm_id = diag.hadm_id
        GROUP BY ad.hadm_id
    )
    , ag AS
    (
        SELECT 
            hadm_id
            , age
            , CASE WHEN age <= 40 THEN 0
        WHEN age <= 50 THEN 1
        WHEN age <= 60 THEN 2
        WHEN age <= 70 THEN 3
        ELSE 4 END AS age_score
        FROM mimic_derived.age
    )
    SELECT 
        ad.subject_id
        , ad.hadm_id
        , ag.age_score
        , myocardial_infarct
        , congestive_heart_failure
        , peripheral_vascular_disease
        , cerebrovascular_disease
        , dementia
        , chronic_pulmonary_disease
        , rheumatic_disease
        , peptic_ulcer_disease
        , mild_liver_disease
        , diabetes_without_cc
        , diabetes_with_cc
        , paraplegia
        , renal_disease
        , malignant_cancer
        , severe_liver_disease 
        , metastatic_solid_tumor 
        , aids
        -- Calculate the Charlson Comorbidity Score using the original
        -- weights from Charlson, 1987.
        , age_score
        + myocardial_infarct + congestive_heart_failure + peripheral_vascular_disease
        + cerebrovascular_disease + dementia + chronic_pulmonary_disease
        + rheumatic_disease + peptic_ulcer_disease
        + GREATEST(mild_liver_disease, 3*severe_liver_disease)
        + GREATEST(2*diabetes_with_cc, diabetes_without_cc)
        + GREATEST(2*malignant_cancer, 6*metastatic_solid_tumor)
        + 2*paraplegia + 2*renal_disease 
        + 6*aids
        AS charlson_comorbidity_index
    FROM mimic_core.admissions ad
    LEFT JOIN com
    ON ad.hadm_id = com.hadm_id
    LEFT JOIN ag
    ON com.hadm_id = ag.hadm_id

    ;
    '''

    cci = pd.read_sql(query, con)

Wall time: 2min 5s


In [19]:
if mode == 'query':
    with open('cci_score.pkl', 'wb') as f:
        pkl.dump(cci, f)
elif mode == 'load':
    with open('cci_score.pkl', 'rb') as f:
        cci = pkl.load(f)

# SOFA

In [56]:
query = '''
WITH co AS
(
  select ih.stay_id, ie.hadm_id
  , hr
  -- start/endtime can be used to filter to values within this hour
  , DATETIME_SUB(ih.endtime, INTERVAL '1' HOUR) AS starttime
  , ih.endtime
  from mimic_derived.icustay_hourly ih
  INNER JOIN mimic_icu.icustays ie
    ON ih.stay_id = ie.stay_id
)
, pafi as
(
  -- join blood gas to ventilation durations to determine if patient was vent
  select ie.stay_id
  , bg.charttime
  -- because pafi has an interaction between vent/PaO2:FiO2, we need two columns for the score
  -- it can happen that the lowest unventilated PaO2/FiO2 is 68, but the lowest ventilated PaO2/FiO2 is 120
  -- in this case, the SOFA score is 3, *not* 4.
  , case when vd.stay_id is null then pao2fio2ratio else null end pao2fio2ratio_novent
  , case when vd.stay_id is not null then pao2fio2ratio else null end pao2fio2ratio_vent
  FROM mimic_icu.icustays ie
  inner join mimic_derived.bg bg
    on ie.subject_id = bg.subject_id
  left join mimic_derived.ventilator_durations vd
    on ie.stay_id = vd.stay_id
    and bg.charttime >= vd.starttime
    and bg.charttime <= vd.endtime
  WHERE specimen_pred = 'ART.'
)
, vs AS
(
    
  select co.stay_id, co.hr
  -- vitals
  , min(vs.mbp) as meanbp_min
  from co
  left join mimic_derived.vitalsign vs
    on co.stay_id = vs.stay_id
    and co.starttime < vs.charttime
    and co.endtime >= vs.charttime
  group by co.stay_id, co.hr
)
, gcs AS
(
  select co.stay_id, co.hr
  -- gcs
  , min(gcs.gcs) as gcs_min
  from co
  left join mimic_derived.gcs gcs
    on co.stay_id = gcs.stay_id
    and co.starttime < gcs.charttime
    and co.endtime >= gcs.charttime
  group by co.stay_id, co.hr
)
, bili AS
(
  select co.stay_id, co.hr
  , max(enz.bilirubin_total) as bilirubin_max
  from co
  left join mimic_derived.enzyme enz
    on co.hadm_id = enz.hadm_id
    and co.starttime < enz.charttime
    and co.endtime >= enz.charttime
  group by co.stay_id, co.hr
)
, cr AS
(
  select co.stay_id, co.hr
  , max(chem.creatinine) as creatinine_max
  from co
  left join mimic_derived.chemistry chem
    on co.hadm_id = chem.hadm_id
    and co.starttime < chem.charttime
    and co.endtime >= chem.charttime
  group by co.stay_id, co.hr
)
, plt AS
(
  select co.stay_id, co.hr
  , min(cbc.platelet) as platelet_min
  from co
  left join mimic_derived.complete_blood_count cbc
    on co.hadm_id = cbc.hadm_id
    and co.starttime < cbc.charttime
    and co.endtime >= cbc.charttime
  group by co.stay_id, co.hr
)
, pf AS
(
  select co.stay_id, co.hr
  , min(pafi.pao2fio2ratio_novent) AS pao2fio2ratio_novent
  , min(pafi.pao2fio2ratio_vent) AS pao2fio2ratio_vent
  from co
  -- bring in blood gases that occurred during this hour
  left join pafi
    on co.stay_id = pafi.stay_id
    and co.starttime < pafi.charttime
    and co.endtime  >= pafi.charttime
  group by co.stay_id, co.hr
)
-- sum uo separately to prevent duplicating values
, uo as
(
  select co.stay_id, co.hr
  -- uo
  , MAX(
      CASE WHEN uo.uo_tm_24hr >= 22 AND uo.uo_tm_24hr <= 30
          THEN uo.urineoutput_24hr / uo.uo_tm_24hr * 24
  END) as uo_24hr
  from co
  left join mimic_derived.urine_output_rate uo
    on co.stay_id = uo.stay_id
    and co.starttime < uo.charttime
    and co.endtime >= uo.charttime
  group by co.stay_id, co.hr
)
-- collapse vasopressors into 1 row per hour
-- also ensures only 1 row per chart time
, vaso AS
(
    SELECT 
        co.stay_id
        , co.hr
        , MAX(epi.vaso_rate) as rate_epinephrine
        , MAX(nor.vaso_rate) as rate_norepinephrine
        , MAX(dop.vaso_rate) as rate_dopamine
        , MAX(dob.vaso_rate) as rate_dobutamine
    FROM co
    LEFT JOIN mimic_derived.epinephrine epi
        on co.stay_id = epi.stay_id
        and co.endtime > epi.starttime
        and co.endtime <= epi.endtime
    LEFT JOIN mimic_derived.norepinephrine nor
        on co.stay_id = nor.stay_id
        and co.endtime > nor.starttime
        and co.endtime <= nor.endtime
    LEFT JOIN mimic_derived.dopamine dop
        on co.stay_id = dop.stay_id
        and co.endtime > dop.starttime
        and co.endtime <= dop.endtime
    LEFT JOIN mimic_derived.dobutamine dob
        on co.stay_id = dob.stay_id
        and co.endtime > dob.starttime
        and co.endtime <= dob.endtime
    WHERE epi.stay_id IS NOT NULL
    OR nor.stay_id IS NOT NULL
    OR dop.stay_id IS NOT NULL
    OR dob.stay_id IS NOT NULL
    GROUP BY co.stay_id, co.hr
)
, scorecomp as
(
  select
      co.stay_id
    , co.hr
    , co.starttime, co.endtime
    , pf.pao2fio2ratio_novent
    , pf.pao2fio2ratio_vent
    , vaso.rate_epinephrine
    , vaso.rate_norepinephrine
    , vaso.rate_dopamine
    , vaso.rate_dobutamine
    , vs.meanbp_min
    , gcs.gcs_min
    -- uo
    , uo.uo_24hr
    -- labs
    , bili.bilirubin_max
    , cr.creatinine_max
    , plt.platelet_min
  from co
  left join vs
    on co.stay_id = vs.stay_id
    and co.hr = vs.hr
  left join gcs
    on co.stay_id = gcs.stay_id
    and co.hr = gcs.hr
  left join bili
    on co.stay_id = bili.stay_id
    and co.hr = bili.hr
  left join cr
    on co.stay_id = cr.stay_id
    and co.hr = cr.hr
  left join plt
    on co.stay_id = plt.stay_id
    and co.hr = plt.hr
  left join pf
    on co.stay_id = pf.stay_id
    and co.hr = pf.hr
  left join uo
    on co.stay_id = uo.stay_id
    and co.hr = uo.hr
  left join vaso
    on co.stay_id = vaso.stay_id
    and co.hr = vaso.hr
)
, scorecalc as
(
  -- Calculate the final score
  -- note that if the underlying data is missing, the component is null
  -- eventually these are treated as 0 (normal), but knowing when data is missing is useful for debugging
  select scorecomp.*
  -- Respiration
  , case
      when pao2fio2ratio_vent   < 100 then 4
      when pao2fio2ratio_vent   < 200 then 3
      when pao2fio2ratio_novent < 300 then 2
      when pao2fio2ratio_vent   < 300 then 2
      when pao2fio2ratio_novent < 400 then 1
      when pao2fio2ratio_vent   < 400 then 1
      when coalesce(pao2fio2ratio_vent, pao2fio2ratio_novent) is null then null
      else 0
    end as respiration

  -- Coagulation
  , case
      when platelet_min < 20  then 4
      when platelet_min < 50  then 3
      when platelet_min < 100 then 2
      when platelet_min < 150 then 1
      when platelet_min is null then null
      else 0
    end as coagulation

  -- Liver
  , case
      -- Bilirubin checks in mg/dL
        when bilirubin_max >= 12.0 then 4
        when bilirubin_max >= 6.0  then 3
        when bilirubin_max >= 2.0  then 2
        when bilirubin_max >= 1.2  then 1
        when bilirubin_max is null then null
        else 0
      end as liver

  -- Cardiovascular
  , case
      when rate_dopamine > 15 or rate_epinephrine >  0.1 or rate_norepinephrine >  0.1 then 4
      when rate_dopamine >  5 or rate_epinephrine <= 0.1 or rate_norepinephrine <= 0.1 then 3
      when rate_dopamine >  0 or rate_dobutamine > 0 then 2
      when meanbp_min < 70 then 1
      when coalesce(meanbp_min, rate_dopamine, rate_dobutamine, rate_epinephrine, rate_norepinephrine) is null then null
      else 0
    end as cardiovascular

  -- Neurological failure (GCS)
  , case
      when (gcs_min >= 13 and gcs_min <= 14) then 1
      when (gcs_min >= 10 and gcs_min <= 12) then 2
      when (gcs_min >=  6 and gcs_min <=  9) then 3
      when  gcs_min <   6 then 4
      when  gcs_min is null then null
      else 0
    end as cns

  -- Renal failure - high creatinine or low urine output
  , case
    when (creatinine_max >= 5.0) then 4
    when uo_24hr < 200 then 4
    when (creatinine_max >= 3.5 and creatinine_max < 5.0) then 3
    when uo_24hr < 500 then 3
    when (creatinine_max >= 2.0 and creatinine_max < 3.5) then 2
    when (creatinine_max >= 1.2 and creatinine_max < 2.0) then 1
    when coalesce (uo_24hr, creatinine_max) is null then null
    else 0 
  end as renal
  from scorecomp
)
, score_final as
(
  select s.*
    -- Combine all the scores to get SOFA
    -- Impute 0 if the score is missing
   -- the window function takes the max over the last 24 hours
    , coalesce(
        MAX(respiration) OVER (PARTITION BY stay_id ORDER BY HR
        ROWS BETWEEN 23 PRECEDING AND 0 FOLLOWING)
      ,0) as respiration_24hours
     , coalesce(
         MAX(coagulation) OVER (PARTITION BY stay_id ORDER BY HR
         ROWS BETWEEN 23 PRECEDING AND 0 FOLLOWING)
        ,0) as coagulation_24hours
    , coalesce(
        MAX(liver) OVER (PARTITION BY stay_id ORDER BY HR
        ROWS BETWEEN 23 PRECEDING AND 0 FOLLOWING)
      ,0) as liver_24hours
    , coalesce(
        MAX(cardiovascular) OVER (PARTITION BY stay_id ORDER BY HR
        ROWS BETWEEN 23 PRECEDING AND 0 FOLLOWING)
      ,0) as cardiovascular_24hours
    , coalesce(
        MAX(cns) OVER (PARTITION BY stay_id ORDER BY HR
        ROWS BETWEEN 23 PRECEDING AND 0 FOLLOWING)
      ,0) as cns_24hours
    , coalesce(
        MAX(renal) OVER (PARTITION BY stay_id ORDER BY HR
        ROWS BETWEEN 23 PRECEDING AND 0 FOLLOWING)
      ,0) as renal_24hours

    -- sum together data for final SOFA
    , coalesce(
        MAX(respiration) OVER (PARTITION BY stay_id ORDER BY HR
        ROWS BETWEEN 23 PRECEDING AND 0 FOLLOWING)
      ,0)
     + coalesce(
         MAX(coagulation) OVER (PARTITION BY stay_id ORDER BY HR
         ROWS BETWEEN 23 PRECEDING AND 0 FOLLOWING)
      ,0)
     + coalesce(
        MAX(liver) OVER (PARTITION BY stay_id ORDER BY HR
        ROWS BETWEEN 23 PRECEDING AND 0 FOLLOWING)
      ,0)
     + coalesce(
        MAX(cardiovascular) OVER (PARTITION BY stay_id ORDER BY HR
        ROWS BETWEEN 23 PRECEDING AND 0 FOLLOWING)
      ,0)
     + coalesce(
        MAX(cns) OVER (PARTITION BY stay_id ORDER BY HR
        ROWS BETWEEN 23 PRECEDING AND 0 FOLLOWING)
      ,0)
     + coalesce(
        MAX(renal) OVER (PARTITION BY stay_id ORDER BY HR
        ROWS BETWEEN 23 PRECEDING AND 0 FOLLOWING)
      ,0)
    as sofa_24hours
  from scorecalc s
  WINDOW W as
  (
    PARTITION BY stay_id
    ORDER BY hr
    ROWS BETWEEN 23 PRECEDING AND 0 FOLLOWING
  )
)
select * from score_final
where hr >= 0;
'''

sofa = pd.read_sql(query, con)

In [20]:
if mode == 'query':
    with open('sofa_score.pkl', 'wb') as f:
        pkl.dump(sofa, f)
elif mode == 'load':
    with open('sofa_score.pkl', 'rb') as f:
        sofa = pkl.load(f)

In [21]:
sofa.head(10)

,stay_id,hr,starttime,endtime,pao2fio2ratio_novent,pao2fio2ratio_vent,rate_epinephrine,rate_norepinephrine,rate_dopamine,rate_dobutamine,meanbp_min,gcs_min,uo_24hr,bilirubin_max,creatinine_max,platelet_min,respiration,coagulation,liver,cardiovascular,cns,renal,respiration_24hours,coagulation_24hours,liver_24hours,cardiovascular_24hours,cns_24hours,renal_24hours,sofa_24hours
0,30000010,0,2166-12-20 12:00:00,2166-12-20 13:00:00,NaN,91.0,NaN,NaN,NaN,NaN,50.0,3.0,NaN,NaN,4.6,209.0,4.0,0.0,NaN,1.0,4.0,3.0,4,0,0,1,4,4,13
1,30000010,1,2166-12-20 13:00:00,2166-12-20 14:00:00,NaN,285.0,NaN,NaN,NaN,NaN,54.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1.0,NaN,NaN,4,0,0,1,4,4,13
2,30000010,2,2166-12-20 14:00:00,2166-12-20 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,71.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,4,0,0,1,4,4,13
3,30000010,3,2166-12-20 15:00:00,2166-12-20 16:00:00,NaN,230.0,NaN,NaN,NaN,NaN,58.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1.0,NaN,NaN,4,0,0,1,4,4,13
4,30000010,4,2166-12-20 16:00:00,2166-12-20 17:00:00,NaN,NaN,NaN,NaN,NaN,NaN,74.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,4,0,0,1,4,4,13
5,30000010,5,2166-12-20 17:00:00,2166-12-20 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,4,0,0,1,4,4,13
6,30000010,6,2166-12-20 18:00:00,2166-12-20 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,57.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,4,0,0,1,4,4,13
7,30000010,7,2166-12-20 19:00:00,2166-12-20 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,63.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,4,0,0,1,4,4,13
8,30000010,8,2166-12-20 20:00:00,2166-12-20 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,61.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,4,0,0,1,4,4,13
9,30000010,9,2166-12-20 21:00:00,2166-12-20 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,70.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,4,0,0,1,4,4,13


In [52]:
sofa_total = sofa.groupby('stay_id').agg({'hr':'count'}).reset_index()

In [29]:
sofa['date'] = sofa.starttime.dt.date

In [30]:
sofa_on_icu_adm = sofa[sofa['hr']==0]

In [31]:
## worst and variability of sofa per 24h
sofa_daily = sofa.groupby(['stay_id','date']).agg({'sofa_24hours':['max','std','mean']}).reset_index() #worst and std of sofa 24h per day
sofa_daily.head(10)

stay_id        date sofa_24hours                     
                                 max       std       mean
0  30000010  2166-12-20           13  0.000000  13.000000
1  30000010  2166-12-21           13  3.378320   9.750000
2  30000010  2166-12-22            7  0.494535   6.375000
3  30000010  2166-12-23            8  1.020621   7.208333
4  30000010  2166-12-24            8  0.000000   8.000000
5  30000010  2166-12-25            8  0.282330   7.083333
6  30000010  2166-12-26            7  0.717282   6.416667
7  30000010  2166-12-27            8  0.337832   7.875000
8  30000010  2166-12-28            9  0.204124   8.041667
9  30000010  2166-12-29           10  0.858673   9.291667

In [32]:
sofa_daily.columns = [''.join(col).strip() for col in sofa_daily.columns.values]

In [33]:
sofa_daily.columns

Index(['stay_id', 'date', 'sofa_24hoursmax', 'sofa_24hoursstd',
       'sofa_24hoursmean'],
      dtype='object')

In [34]:
sofa_daily['sofa_24hoursmean_diff'] = sofa_daily.groupby(['stay_id'])['sofa_24hoursmean'].diff()

In [35]:
sofa_daily.head(30)

,stay_id,date,sofa_24hoursmax,sofa_24hoursstd,sofa_24hoursmean,sofa_24hoursmean_diff
0,30000010,2166-12-20,13,0.000000,13.000000,NaN
1,30000010,2166-12-21,13,3.378320,9.750000,-3.250000
2,30000010,2166-12-22,7,0.494535,6.375000,-3.375000
3,30000010,2166-12-23,8,1.020621,7.208333,0.833333
4,30000010,2166-12-24,8,0.000000,8.000000,0.791667
5,30000010,2166-12-25,8,0.282330,7.083333,-0.916667
6,30000010,2166-12-26,7,0.717282,6.416667,-0.666667
7,30000010,2166-12-27,8,0.337832,7.875000,1.458333
8,30000010,2166-12-28,9,0.204124,8.041667,0.166667
9,30000010,2166-12-29,10,0.858673,9.291667,1.250000


In [36]:
sofa_daily_agg = sofa_daily.groupby('stay_id').agg({'sofa_24hoursmax':'max','sofa_24hoursstd':'mean','sofa_24hoursmean':'mean',\
                                   'sofa_24hoursmean_diff':['min','mean']}).reset_index()

In [37]:
sofa_daily_agg.columns = [''.join(col).strip() for col in sofa_daily_agg.columns.values]

In [155]:
sofa_daily_agg.head()

,stay_id,sofa_24hoursmaxmax,sofa_24hoursstdmean,sofa_24hoursmeanmean,sofa_24hoursmean_diffmin,sofa_24hoursmean_diffmean
0,30000010,13,0.727003,7.824248,-3.375000,-5.546559e-01
1,30000186,5,0.238366,4.340909,0.681818,6.818182e-01
2,30000575,3,0.474829,2.710317,-0.583333,-1.488095e-01
3,30000670,6,0.775251,1.277778,-2.916667,4.510281e-17
4,30000974,6,0.481713,2.325980,-4.083333,-4.687500e-02


In [39]:
## worst and average of sofa for stay
sofa.groupby(['stay_id']).agg({'sofa_24hours':['max','mean']})

sofa_24hours          
                  max      mean
stay_id                        
30000010           13  7.661442
30000186            5  4.535714
30000575            3  2.790698
30000670            6  1.437500
30000974            6  2.305211
...               ...       ...
39998269           17  9.960920
39998606            2  1.636364
39998664            1  0.800000
39998706            7  6.692308
39999149            3  2.476190

[69607 rows x 2 columns]

# Merge the dataframes together

In [40]:
### making sure we only have index icu stays

In [41]:
num_stay = hosp_icu.groupby('hadm_id').agg({'stay_id':'nunique'}).reset_index()

In [42]:
num_stay[num_stay['stay_id']>1] 

,hadm_id,stay_id


## Merge

In [107]:
hosp_icu.shape, vsvd1.shape, cci.shape, sofa_daily_agg.shape

((58101, 22), (4778, 3), (524520, 21), (69607, 6))

In [134]:
master = pd.merge(pd.merge(pd.merge(hosp_icu, vsvd1, on=['subject_id','stay_id']), cci, on=['subject_id','hadm_id']),\
                  sofa_daily_agg, on='stay_id')

In [135]:
master['bef_icu_los'] = (master['intime'] - master['admittime']).dt.days

In [136]:
## create label
master['icu_disch_los'] = (master['dischtime'] - master['outtime']).dt.days

In [137]:
master.head()

,gender,anchor_age,dod,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,edregtime,edouttime,hospital_expire_flag,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,icu_los,hosp_los,vent_duration,age_score,myocardial_infarct,congestive_heart_failure,peripheral_vascular_disease,cerebrovascular_disease,dementia,chronic_pulmonary_disease,rheumatic_disease,peptic_ulcer_disease,mild_liver_disease,diabetes_without_cc,diabetes_with_cc,paraplegia,renal_disease,malignant_cancer,severe_liver_disease,metastatic_solid_tumor,aids,charlson_comorbidity_index,sofa_24hoursmaxmax,sofa_24hoursstdmean,sofa_24hoursmeanmean,sofa_24hoursmean_diffmin,sofa_24hoursmean_diffmean,bef_icu_los,icu_disch_los
0,M,72,None,2158-02-13 04:59:00,2158-02-26 14:45:00,None,EW EMER.,EMERGENCY ROOM,REHAB,Medicare,2158-02-13 02:15:00,2158-02-13 03:24:00,0,19836972,20014283,32496266,Coronary Care Unit (CCU),Cardiac Vascular Intensive Care Unit (CVICU),2158-02-13 04:59:54,2158-02-15 18:54:41,2.579711,13.0,1.0,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,6,1.142604,2.747807,2.493421,2.684211,0,10
1,F,81,None,2142-04-09 20:06:00,2142-04-15 16:47:00,None,EW EMER.,PROCEDURE SITE,SKILLED NURSING FACILITY,Medicare,NaT,NaT,0,16651226,20026217,35802155,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2142-04-10 14:21:57,2142-04-11 21:08:28,1.282303,5.0,1.0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,10,1.587623,7.385714,2.371429,2.371429,0,3
2,M,38,None,2145-02-07 21:02:00,2145-02-14 16:08:00,None,EW EMER.,EMERGENCY ROOM,SKILLED NURSING FACILITY,Other,2145-02-07 18:31:00,2145-02-07 22:00:00,0,10291458,20046372,37588913,Trauma SICU (TSICU),Trauma SICU (TSICU),2145-02-07 22:00:00,2145-02-11 11:06:39,3.546285,6.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0.538778,2.300000,-3.541667,-0.250000,0,3
3,F,23,None,2173-05-19 23:58:00,2173-05-23 14:00:00,None,EW EMER.,EMERGENCY ROOM,HOME,Other,2173-05-19 20:23:00,2173-05-20 02:42:00,0,10922424,20057730,34246418,Trauma SICU (TSICU),Trauma SICU (TSICU),2173-05-20 02:42:00,2173-05-21 00:10:29,0.894780,3.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0.300793,1.904762,NaN,NaN,0,2
4,F,82,None,2164-09-01 23:14:00,2164-09-04 17:30:00,None,EW EMER.,EMERGENCY ROOM,SKILLED NURSING FACILITY,Other,2164-09-01 16:51:00,2164-09-02 01:05:00,0,11549236,20060563,36714996,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2164-09-02 01:05:00,2164-09-02 19:29:36,0.767083,2.0,1.0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,1,0.235702,0.944444,NaN,NaN,0,1


In [138]:
master.shape

(3980, 49)

In [139]:
master[master['sofa_24hoursmaxmax'].isna()]

,gender,anchor_age,dod,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,edregtime,edouttime,hospital_expire_flag,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,icu_los,hosp_los,vent_duration,age_score,myocardial_infarct,congestive_heart_failure,peripheral_vascular_disease,cerebrovascular_disease,dementia,chronic_pulmonary_disease,rheumatic_disease,peptic_ulcer_disease,mild_liver_disease,diabetes_without_cc,diabetes_with_cc,paraplegia,renal_disease,malignant_cancer,severe_liver_disease,metastatic_solid_tumor,aids,charlson_comorbidity_index,sofa_24hoursmaxmax,sofa_24hoursstdmean,sofa_24hoursmeanmean,sofa_24hoursmean_diffmin,sofa_24hoursmean_diffmean,bef_icu_los,icu_disch_los


### Master dataframe post-cleaning

In [140]:
master = master[~master['discharge_location'].isin(['DIED','AGAINST ADVICE','ACUTE HOSPITAL','HEALTHCARE FACILITY'])]

In [141]:
master = master.replace({'icu_disch_los':{-1:0}})

In [142]:
master.shape

(3920, 49)

# Model

In [143]:
features = ['anchor_age','bef_icu_los','icu_los','vent_duration',\
'age_score','myocardial_infarct','congestive_heart_failure','peripheral_vascular_disease',\
'cerebrovascular_disease','dementia','chronic_pulmonary_disease','rheumatic_disease','peptic_ulcer_disease',\
'mild_liver_disease','diabetes_without_cc','diabetes_with_cc','paraplegia','renal_disease','malignant_cancer',\
'severe_liver_disease','metastatic_solid_tumor','aids','charlson_comorbidity_index','sofa_24hoursmaxmax',\
'sofa_24hoursstdmean','sofa_24hoursmeanmean','sofa_24hoursmean_diffmin','sofa_24hoursmean_diffmean']

label = ['icu_disch_los']

In [144]:
df = master[features+label]
X = master[features]
y = master[label]

In [145]:
df.head()

,anchor_age,bef_icu_los,icu_los,vent_duration,age_score,myocardial_infarct,congestive_heart_failure,peripheral_vascular_disease,cerebrovascular_disease,dementia,chronic_pulmonary_disease,rheumatic_disease,peptic_ulcer_disease,mild_liver_disease,diabetes_without_cc,diabetes_with_cc,paraplegia,renal_disease,malignant_cancer,severe_liver_disease,metastatic_solid_tumor,aids,charlson_comorbidity_index,sofa_24hoursmaxmax,sofa_24hoursstdmean,sofa_24hoursmeanmean,sofa_24hoursmean_diffmin,sofa_24hoursmean_diffmean,icu_disch_los
0,72,0,2.579711,1.0,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,6,1.142604,2.747807,2.493421,2.684211,10
1,81,0,1.282303,1.0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,10,1.587623,7.385714,2.371429,2.371429,3
2,38,0,3.546285,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0.538778,2.300000,-3.541667,-0.250000,3
3,23,0,0.894780,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0.300793,1.904762,NaN,NaN,2
4,82,0,0.767083,1.0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,1,0.235702,0.944444,NaN,NaN,1


In [146]:
from sklearn.model_selection import train_test_split

In [147]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [148]:
from sklearn.preprocessing import MinMaxScaler

In [149]:
scaler = MinMaxScaler()

In [150]:
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), index=X_train.index, columns=X_train.columns)

In [151]:
X_train_scaled = X_train_scaled.fillna(-999)

In [152]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [153]:
X_train_scaled2 = sm.add_constant(X_train_scaled)
est = sm.OLS(y_train, X_train_scaled2)
est2 = est.fit()
est2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          icu_disch_los   R-squared:                       0.099
Model:                            OLS   Adj. R-squared:                  0.091
Method:                 Least Squares   F-statistic:                     12.21
Date:                Sun, 13 Dec 2020   Prob (F-statistic):           1.78e-52
Time:                        14:13:53   Log-Likelihood:                -10127.
No. Observations:                3136   AIC:                         2.031e+04
Df Residuals:                    3107   BIC:                         2.049e+04
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                           4.4489      0.494      9.011      0.000       3.481       5.417
anchor_age                     -0.3269      1.593     -0.205      0.837      -3.451       2.797
bef_icu_los                    16.9970      2.413      7.042      0.000      12.265      21.729
icu_los                         6.4344      1.594      4.037      0.000       3.309       9.560
vent_duration                   4.1349      1.270      3.255      0.001       1.645       6.625
age_score                       2.9446      2.629      1.120      0.263      -2.211       8.100
myocardial_infarct              0.8627      0.682      1.265      0.206      -0.474       2.200
congestive_heart_failure        0.6798      0.664      1.023      0.306      -0.623       1.983
peripheral_vascular_disease     1.1863      0.685      1.731      0.083      -0.157       2.530
cerebrovascular_disease         2.0296      0.712      2.851      0.004       0.634       3.426
dementia                        2.5597      1.082      2.366      0.018       0.438       4.681
chronic_pulmonary_disease       0.8672      0.651      1.332      0.183      -0.410       2.144
rheumatic_disease               2.0561      0.887      2.318      0.020       0.317       3.795
peptic_ulcer_disease            4.4507      0.947      4.698      0.000       2.593       6.308
mild_liver_disease              1.6417      0.745      2.203      0.028       0.181       3.103
diabetes_without_cc             0.5253      0.614      0.855      0.392      -0.679       1.729
diabetes_with_cc                2.6203      1.159      2.260      0.024       0.347       4.894
paraplegia                      3.9133      1.375      2.847      0.004       1.218       6.609
renal_disease                   2.3882      1.243      1.921      0.055      -0.049       4.826
malignant_cancer                3.5431      1.150      3.081      0.002       1.288       5.798
severe_liver_disease            2.1912      1.462      1.499      0.134      -0.675       5.057
metastatic_solid_tumor          7.2101      2.991      2.411      0.016       1.346      13.075
aids                           11.7940      4.030      2.927      0.003       3.892      19.696
charlson_comorbidity_index    -18.6667     10.058     -1.856      0.064     -38.389       1.055
sofa_24hoursmaxmax             -9.9397      2.526     -3.935      0.000     -14.893      -4.987
sofa_24hoursstdmean            -0.8100      1.045     -0.775      0.438      -2.859       1.239
sofa_24hoursmeanmean           10.1739      1.960      5.191      0.000       6.331      14.017
sofa_24hoursmean_diffmin       -9.5395      1.937     -4.924      0.000     -13.338      -5.741
sofa_24hoursmean_diffmean       9.5421      1.937      4.925      0.000       5.743      13.341
===========

In [154]:
# regularized OLS
model = sm.OLS(y_train, X_train_scaled2)
est = model.fit()
est_reg = model.fit_regularized(L1_wt=0.05,alpha = 0.03) # L1_wt=0 - ridge L1_wt=1 lasso (needs package cvxopt)
est_fit = sm.regression.linear_model.OLSResults(est_reg.model,est_reg.params,est.normalized_cov_params)
est_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          icu_disch_los   R-squared:                       0.062
Model:                            OLS   Adj. R-squared:                  0.054
Method:                 Least Squares   F-statistic:                     7.372
Date:                Sun, 13 Dec 2020   Prob (F-statistic):           5.09e-28
Time:                        14:14:15   Log-Likelihood:                -10190.
No. Observations:                3136   AIC:                         2.044e+04
Df Residuals:                    3107   BIC:                         2.061e+04
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                           3.5816      0.504      7.111      0.000       2.594       4.569
anchor_age                           0      1.625          0      1.000      -3.187       3.187
bef_icu_los                     1.2294      2.462      0.499      0.618      -3.599       6.057
icu_los                         2.0938      1.626      1.288      0.198      -1.095       5.282
vent_duration                   1.7855      1.296      1.378      0.168      -0.755       4.326
age_score                      -0.8511      2.682     -0.317      0.751      -6.111       4.408
myocardial_infarct             -0.2520      0.696     -0.362      0.717      -1.616       1.112
congestive_heart_failure       -0.3613      0.678     -0.533      0.594      -1.691       0.968
peripheral_vascular_disease          0      0.699          0      1.000      -1.371       1.371
cerebrovascular_disease         0.9349      0.726      1.287      0.198      -0.489       2.359
dementia                        0.3333      1.104      0.302      0.763      -1.831       2.498
chronic_pulmonary_disease      -0.1251      0.664     -0.188      0.851      -1.428       1.178
rheumatic_disease               0.4272      0.905      0.472      0.637      -1.347       2.201
peptic_ulcer_disease            1.6345      0.967      1.691      0.091      -0.261       3.530
mild_liver_disease              0.7734      0.760      1.017      0.309      -0.717       2.264
diabetes_without_cc            -0.4084      0.626     -0.652      0.515      -1.637       0.820
diabetes_with_cc                0.4301      1.183      0.364      0.716      -1.889       2.749
paraplegia                      0.9499      1.403      0.677      0.498      -1.800       3.700
renal_disease                   0.2351      1.268      0.185      0.853      -2.252       2.722
malignant_cancer                1.2512      1.173      1.066      0.286      -1.049       3.552
severe_liver_disease            0.4421      1.491      0.296      0.767      -2.482       3.366
metastatic_solid_tumor          0.9735      3.052      0.319      0.750      -5.010       6.957
aids                            0.4749      4.112      0.115      0.908      -7.587       8.537
charlson_comorbidity_index      0.6861     10.262      0.067      0.947     -19.435      20.807
sofa_24hoursmaxmax              1.7062      2.577      0.662      0.508      -3.347       6.760
sofa_24hoursstdmean             0.1138      1.066      0.107      0.915      -1.976       2.204
sofa_24hoursmeanmean            1.3602      2.000      0.680      0.496      -2.560       5.281
sofa_24hoursmean_diffmin        0.0019      1.977      0.001      0.999      -3.874       3.877
sofa_24hoursmean_diffmean       0.0001      1.977   6.13e-05      1.000      -3.876       3.876
===========

## Sepsis

In [70]:
query = '''
WITH ab_tbl AS 
(
  select
      abx.subject_id, abx.hadm_id, abx.stay_id
    , abx.antibiotic
    , abx.starttime AS antibiotic_time
    -- date is used to match microbiology cultures with only date available
    , abx.starttime AS antibiotic_date
    , abx.stoptime AS antibiotic_stoptime
    -- create a unique identifier for each patient antibiotic
    , ROW_NUMBER() OVER
    (
      PARTITION BY subject_id
      ORDER BY starttime, stoptime, antibiotic
    ) AS ab_id
  from mimic_derived.antibiotic abx
)
, me as
(
  select micro_specimen_id
    -- the following columns are identical for all rows of the same micro_specimen_id
    -- these aggregates simply collapse duplicates down to 1 row
    , MAX(subject_id) AS subject_id
    , MAX(hadm_id) AS hadm_id
    , CAST(MAX(chartdate) AS DATE) AS chartdate
    , MAX(charttime) AS charttime
    , MAX(spec_type_desc) AS spec_type_desc
    , max(case when org_name is not null and org_name != '' then 1 else 0 end) as PositiveCulture
  from mimic_hosp.microbiologyevents
  group by micro_specimen_id
)
-- culture followed by an antibiotic
, me_then_ab AS
(
  select
    ab_tbl.subject_id
    , ab_tbl.hadm_id
    , ab_tbl.stay_id
    , ab_tbl.ab_id
    
    , me72.micro_specimen_id
    , coalesce(me72.charttime, CAST(me72.chartdate AS DATETIME)) as last72_charttime
    , me72.positiveculture as last72_positiveculture
    , me72.spec_type_desc as last72_specimen

    -- we will use this partition to select the earliest culture before this abx
    -- this ensures each antibiotic is only matched to a single culture
    -- and consequently we have 1 row per antibiotic
    , ROW_NUMBER() OVER
    (
      PARTITION BY ab_tbl.subject_id, ab_tbl.ab_id
      ORDER BY me72.chartdate, me72.charttime NULLS LAST
    ) AS micro_seq
  from ab_tbl
  -- abx taken after culture, but no more than 72 hours after
  LEFT JOIN me me72
    on ab_tbl.subject_id = me72.subject_id
    and
    (
      (
      -- if charttime is available, use it
          me72.charttime is not null
      and ab_tbl.antibiotic_time > me72.charttime
      and ab_tbl.antibiotic_time <= DATETIME_ADD(me72.charttime, 'INTERVAL 72 HOUR') 
      )
      OR
      (
      -- if charttime is not available, use chartdate
          me72.charttime is null
      and antibiotic_date >= me72.chartdate
      and antibiotic_date <= DATE_ADD(me72.chartdate, 'INTERVAL 3 DAY')
      )
    )
)
, ab_then_me AS
(
  select
      ab_tbl.subject_id
    , ab_tbl.hadm_id
    , ab_tbl.stay_id
    , ab_tbl.ab_id
    
    , me24.micro_specimen_id
    , COALESCE(me24.charttime, CAST(me24.chartdate AS DATETIME)) as next24_charttime
    , me24.positiveculture as next24_positiveculture
    , me24.spec_type_desc as next24_specimen

    -- we will use this partition to select the earliest culture before this abx
    -- this ensures each antibiotic is only matched to a single culture
    -- and consequently we have 1 row per antibiotic
    , ROW_NUMBER() OVER
    (
      PARTITION BY ab_tbl.subject_id, ab_tbl.ab_id
      ORDER BY me24.chartdate, me24.charttime NULLS LAST
    ) AS micro_seq
  from ab_tbl
  -- culture in subsequent 24 hours
  LEFT JOIN me me24
    on ab_tbl.subject_id = me24.subject_id
    and
    (
      (
          -- if charttime is available, use it
          me24.charttime is not null
      and ab_tbl.antibiotic_time >= DATETIME_SUB(me24.charttime, 'INTERVAL 24 HOUR')  
      and ab_tbl.antibiotic_time < me24.charttime
      )
      OR
      (
          -- if charttime is not available, use chartdate
          me24.charttime is null
      and ab_tbl.antibiotic_date >= DATE_SUB(me24.chartdate, 'INTERVAL 1 DAY')
      and ab_tbl.antibiotic_date <= me24.chartdate
      )
    )
)
SELECT
ab_tbl.subject_id
, ab_tbl.stay_id
, ab_tbl.hadm_id
, ab_tbl.ab_id
, ab_tbl.antibiotic
, ab_tbl.antibiotic_time

, CASE
  WHEN last72_specimen IS NULL AND next24_specimen IS NULL
    THEN 0
  ELSE 1 
  END AS suspected_infection
-- time of suspected infection:
--    (1) the culture time (if before antibiotic)
--    (2) or the antibiotic time (if before culture)
, CASE
  WHEN last72_specimen IS NULL AND next24_specimen IS NULL
    THEN NULL
  ELSE COALESCE(last72_charttime, antibiotic_time)
  END AS suspected_infection_time

, COALESCE(last72_charttime, next24_charttime) AS culture_time

-- the specimen that was cultured
, COALESCE(last72_specimen, next24_specimen) AS specimen

-- whether the cultured specimen ended up being positive or not
, COALESCE(last72_positiveculture, next24_positiveculture) AS positive_culture

FROM ab_tbl
LEFT JOIN ab_then_me ab2me
    ON ab_tbl.subject_id = ab2me.subject_id
    AND ab_tbl.ab_id = ab2me.ab_id
    AND ab2me.micro_seq = 1
LEFT JOIN me_then_ab me2ab
    ON ab_tbl.subject_id = me2ab.subject_id
    AND ab_tbl.ab_id = me2ab.ab_id
    AND me2ab.micro_seq = 1
;

'''

oas = pd.read_sql(query,con)

DatabaseError: Execution failed on sql '
WITH ab_tbl AS 
(
  select
      abx.subject_id, abx.hadm_id, abx.stay_id
    , abx.antibiotic
    , abx.starttime AS antibiotic_time
    -- date is used to match microbiology cultures with only date available
    , abx.starttime AS antibiotic_date
    , abx.stoptime AS antibiotic_stoptime
    -- create a unique identifier for each patient antibiotic
    , ROW_NUMBER() OVER
    (
      PARTITION BY subject_id
      ORDER BY starttime, stoptime, antibiotic
    ) AS ab_id
  from mimic_derived.antibiotic abx
)
, me as
(
  select micro_specimen_id
    -- the following columns are identical for all rows of the same micro_specimen_id
    -- these aggregates simply collapse duplicates down to 1 row
    , MAX(subject_id) AS subject_id
    , MAX(hadm_id) AS hadm_id
    , CAST(MAX(chartdate) AS DATE) AS chartdate
    , MAX(charttime) AS charttime
    , MAX(spec_type_desc) AS spec_type_desc
    , max(case when org_name is not null and org_name != '' then 1

In [6]:
%%time
# query = 'SELECT * FROM admissions limit 500'

# query = '''
# SELECT *
# FROM mimic_core.patients pat
# JOIN mimic_core.admissions adm on pat.subject_id = adm.subject_id
# JOIN mimic_icu.icustays icu on adm.subject_id = icu.subject_id and adm.hadm_id = icu.hadm_id
# JOIN (select hadm_id  , min(intime) as min_intime FROM mimic_icu.icustays GROUP BY hadm_id ) first_ on icu.hadm_id = first_.hadm_id and icu.intime = first_.min_intime
# --on adm.hadm_id = first_icu.hadm_id
# WHERE  pat.anchor_age >= 18
# AND adm.hospital_expire_flag = 0
# AND icu.stay_id = 30000010
# AND icu.subject_id in (select subject_id from mimic_derived.ventilator_setting where extubated = 1 )
# limit 1000
# '''

# query = '''
# select

# a.stay_id, a.charttime, a.storetime, a.value as device , b.value as mode ,b.valuenum
# from

# (
# select  stay_id,charttime,storetime,value,valuenum from chartevents 
# where itemid  = '223848' 
# and lower(value) in ('drager','avea','pb 7200','sensor medic (hfo)','hamilton')

# ) a

# join
# (
#  select  stay_id,charttime,storetime,value,valuenum from chartevents
# where itemid = '223849'

# ) b

# on a.stay_id = b.stay_id and a.charttime = b.charttime

# '''

# vent = pd.read_sql(query, con)
